<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">
*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

<!--NAVIGATION-->
< [Handling Missing Data](03.04-Missing-Values.ipynb) | [Contents](Index.ipynb) | [Combining Datasets: Concat and Append](03.06-Concat-And-Append.ipynb) >

# Hierarchical Indexing

Up to this point we've been focused primarily on one-dimensional and two-dimensional data, stored in Pandas ``Series`` and ``DataFrame`` objects, respectively.
Often it is useful to go beyond this and store higher-dimensional data–that is, data indexed by more than one or two keys.
While Pandas does provide ``Panel`` and ``Panel4D`` objects that natively handle three-dimensional and four-dimensional data (see [Aside: Panel Data](#Aside:-Panel-Data)), a far more common pattern in practice is to make use of *hierarchical indexing* (also known as *multi-indexing*) to incorporate multiple index *levels* within a single index.
In this way, higher-dimensional data can be compactly represented within the familiar one-dimensional ``Series`` and two-dimensional ``DataFrame`` objects.

In this section, we'll explore the direct creation of ``MultiIndex`` objects, considerations when indexing, slicing, and computing statistics across multiply indexed data, and useful routines for converting between simple and hierarchically indexed representations of your data.

We begin with the standard imports:

In [1]:
import pandas as pd
import numpy as np

## A Multiply Indexed Series

Let's start by considering how we might represent two-dimensional data within a one-dimensional ``Series``.
For concreteness, we will consider a series of data where each point has a character and numerical key.

### The bad way

Suppose you would like to track data about states from two different years.
Using the Pandas tools we've already covered, you might be tempted to simply use Python tuples as keys:

In [2]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

With this indexing scheme, you can straightforwardly index or slice the series based on this multiple index:

In [3]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

But the convenience ends there. For example, if you need to select all values from 2010, you'll need to do some messy (and potentially slow) munging to make it happen:

In [4]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

This produces the desired result, but is not as clean (or as efficient for large datasets) as the slicing syntax we've grown to love in Pandas.

### The Better Way: Pandas MultiIndex
Fortunately, Pandas provides a better way.
Our tuple-based indexing is essentially a rudimentary multi-index, and the Pandas ``MultiIndex`` type gives us the type of operations we wish to have.
We can create a multi-index from the tuples as follows:

In [5]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

Notice that the ``MultiIndex`` contains multiple *levels* of indexing–in this case, the state names and the years, as well as multiple *labels* for each data point which encode these levels.

If we re-index our series with this ``MultiIndex``, we see the hierarchical representation of the data:

In [6]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Here the first two columns of the ``Series`` representation show the multiple index values, while the third column shows the data.
Notice that some entries are missing in the first column: in this multi-index representation, any blank entry indicates the same value as the line above it.

Now to access all data for which the second index is 2010, we can simply use the Pandas slicing notation:

In [7]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

The result is a singly indexed array with just the keys we're interested in.
This syntax is much more convenient (and the operation is much more efficient!) than the home-spun tuple-based multi-indexing solution that we started with.
We'll now further discuss this sort of indexing operation on hieararchically indexed data.

### MultiIndex as extra dimension

You might notice something else here: we could easily have stored the same data using a simple ``DataFrame`` with index and column labels.
In fact, Pandas is built with this equivalence in mind. The ``unstack()`` method will quickly convert a multiply indexed ``Series`` into a conventionally indexed ``DataFrame``:

In [8]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


Naturally, the ``stack()`` method provides the opposite operation:

In [9]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Seeing this, you might wonder why would we would bother with hierarchical indexing at all.
The reason is simple: just as we were able to use multi-indexing to represent two-dimensional data within a one-dimensional ``Series``, we can also use it to represent data of three or more dimensions in a ``Series`` or ``DataFrame``.
Each extra level in a multi-index represents an extra dimension of data; taking advantage of this property gives us much more flexibility in the types of data we can represent. Concretely, we might want to add another column of demographic data for each state at each year (say, population under 18) ; with a ``MultiIndex`` this is as easy as adding another column to the ``DataFrame``:

In [9]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In addition, all the ufuncs and other functionality discussed in [Operating on Data in Pandas](03.03-Operations-in-Pandas.ipynb) work with hierarchical indices as well.
Here we compute the fraction of people under 18 by year, given the above data:

In [10]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


This allows us to easily and quickly manipulate and explore even high-dimensional data.

## Methods of MultiIndex Creation

The most straightforward way to construct a multiply indexed ``Series`` or ``DataFrame`` is to simply pass a list of two or more index arrays to the constructor. For example:

In [12]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
print(df)
print(df.unstack())

        data1     data2
a 1  0.543817  0.535617
  2  0.798506  0.176635
b 1  0.657549  0.683189
  2  0.021702  0.346050
      data1               data2          
          1         2         1         2
a  0.543817  0.798506  0.535617  0.176635
b  0.657549  0.021702  0.683189  0.346050


The work of creating the ``MultiIndex`` is done in the background.

Similarly, if you pass a dictionary with appropriate tuples as keys, Pandas will automatically recognize this and use a ``MultiIndex`` by default:

In [13]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

Nevertheless, it is sometimes useful to explicitly create a ``MultiIndex``; we'll see a couple of these methods here.

### Explicit MultiIndex constructors

For more flexibility in how the index is constructed, you can instead use the class method constructors available in the ``pd.MultiIndex``.
For example, as we did before, you can construct the ``MultiIndex`` from a simple list of arrays giving the index values within each level:

In [15]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

You can construct it from a list of tuples giving the multiple index values of each point:

In [16]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

You can even construct it from a Cartesian product of single indices:

In [31]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2,3]])

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 2),
            ('b', 3)],
           )

Similarly, you can construct the ``MultiIndex`` directly using its internal encoding by passing ``levels`` (a list of lists containing available index values for each level) and ``labels`` (a list of lists that reference these labels):

In [20]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

TypeError: __new__() got an unexpected keyword argument 'labels'

Any of these objects can be passed as the ``index`` argument when creating a ``Series`` or ``Dataframe``, or be passed to the ``reindex`` method of an existing ``Series`` or ``DataFrame``.

### MultiIndex level names

Sometimes it is convenient to name the levels of the ``MultiIndex``.
This can be accomplished by passing the ``names`` argument to any of the above ``MultiIndex`` constructors, or by setting the ``names`` attribute of the index after the fact:

In [21]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

With more involved datasets, this can be a useful way to keep track of the meaning of various index values.

### MultiIndex for columns

In a ``DataFrame``, the rows and columns are completely symmetric, and just as the rows can have multiple levels of indices, the columns can have multiple levels as well.
Consider the following, which is a mock-up of some (somewhat realistic) medical data:

In [51]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014, 2015], [1, 2, 3]],
                                  names=['year', 'session'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp','Admin']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(9,9),4)
data[:, ::20] *= 5
data += 30

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject           Bob                      Guido                        Sue  \
type               HR     Temp    Admin       HR     Temp    Admin       HR   
year session                                                                  
2013 1        36.8280  30.4323  30.2209  29.4441  29.7067  30.2776  30.2611   
     2        26.5965  29.9151  31.8368  31.8484  29.3796  29.0087  27.7863   
     3        39.4260  31.2263  29.1585  29.4538  28.6172  28.9508  29.8821   
2014 1        32.1890  31.4605  30.1290  29.0034  28.7336  29.2168  31.2508   
     2        25.3240  29.3204  28.2928  30.0438  30.4477  30.9551  28.1232   
     3        32.2860  30.1546  28.6942  29.9960  29.6327  29.0865  30.5070   
2015 1        33.1515  29.9863  29.6798  28.3646  31.8518  31.5910  29.3723   
     2        27.9015  32.7687  30.7324  28.7988  29.4542  29.3931  30.4970   
     3        36.8335  29.0620  30.6849  29.8823  27.6963  30.9413  30.5530   

subject                         
type             Temp    Admin  
year session                    
2013 1        27.4274  30.3896  
     2        30.9259  30.1265  
     3        30.8675  30.7979  
2014 1        30.1317  29.0871  
     2        29.9151  29.2973  
     3        28.6844  29.6195  
2015 1        29.8012  30.3921  
     2        27.8455  30.5205  
     3        28.6712  28.8165

In [52]:
index = pd.MultiIndex.from_product([[2013, 2014, 2015], [1, 2, 3]],)
index
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp','Admin']])
columns

MultiIndex([(  'Bob',    'HR'),
            (  'Bob',  'Temp'),
            (  'Bob', 'Admin'),
            ('Guido',    'HR'),
            ('Guido',  'Temp'),
            ('Guido', 'Admin'),
            (  'Sue',    'HR'),
            (  'Sue',  'Temp'),
            (  'Sue', 'Admin')],
           )

Here we see where the multi-indexing for both rows and columns can come in *very* handy.
This is fundamentally four-dimensional data, where the dimensions are the subject, the measurement type, the year, and the visit number.
With this in place we can, for example, index the top-level column by the person's name and get a full ``DataFrame`` containing just that person's information:

In [72]:
health_data[(2013,1):(2015,2)]

subject           Bob                      Guido                        Sue  \
type               HR     Temp    Admin       HR     Temp    Admin       HR   
year session                                                                  
2013 1        36.8280  30.4323  30.2209  29.4441  29.7067  30.2776  30.2611   
     2        26.5965  29.9151  31.8368  31.8484  29.3796  29.0087  27.7863   
     3        39.4260  31.2263  29.1585  29.4538  28.6172  28.9508  29.8821   
2014 1        32.1890  31.4605  30.1290  29.0034  28.7336  29.2168  31.2508   
     2        25.3240  29.3204  28.2928  30.0438  30.4477  30.9551  28.1232   
     3        32.2860  30.1546  28.6942  29.9960  29.6327  29.0865  30.5070   
2015 1        33.1515  29.9863  29.6798  28.3646  31.8518  31.5910  29.3723   
     2        27.9015  32.7687  30.7324  28.7988  29.4542  29.3931  30.4970   

subject                         
type             Temp    Admin  
year session                    
2013 1        27.4274  30.3896  
     2        30.9259  30.1265  
     3        30.8675  30.7979  
2014 1        30.1317  29.0871  
     2        29.9151  29.2973  
     3        28.6844  29.6195  
2015 1        29.8012  30.3921  
     2        27.8455  30.5205

For complicated records containing multiple labeled measurements across multiple times for many subjects (people, countries, cities, etc.) use of hierarchical rows and columns can be extremely convenient!

## Indexing and Slicing a MultiIndex

Indexing and slicing on a ``MultiIndex`` is designed to be intuitive, and it helps if you think about the indices as added dimensions.
We'll first look at indexing multiply indexed ``Series``, and then multiply-indexed ``DataFrame``s.

### Multiply indexed Series

Consider the multiply indexed ``Series`` of state populations we saw earlier:

In [21]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

We can access single elements by indexing with multiple terms:

In [22]:
pop['California', 2000]

33871648

The ``MultiIndex`` also supports *partial indexing*, or indexing just one of the levels in the index.
The result is another ``Series``, with the lower-level indices maintained:

In [23]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

Partial slicing is available as well, as long as the ``MultiIndex`` is sorted (see discussion in [Sorted and Unsorted Indices](#Sorted-and-unsorted-indices)):

In [24]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

With sorted indices, partial indexing can be performed on lower levels by passing an empty slice in the first index:

In [25]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

Other types of indexing and selection (discussed in [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb)) work as well; for example, selection based on Boolean masks:

In [26]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

Selection based on fancy indexing also works:

In [27]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### Multiply indexed DataFrames

A multiply indexed ``DataFrame`` behaves in a similar manner.
Consider our toy medical ``DataFrame`` from before:

In [78]:
health_data

subject           Bob                      Guido                        Sue  \
type               HR     Temp    Admin       HR     Temp    Admin       HR   
year session                                                                  
2013 1        36.8280  30.4323  30.2209  29.4441  29.7067  30.2776  30.2611   
     2        26.5965  29.9151  31.8368  31.8484  29.3796  29.0087  27.7863   
     3        39.4260  31.2263  29.1585  29.4538  28.6172  28.9508  29.8821   
2014 1        32.1890  31.4605  30.1290  29.0034  28.7336  29.2168  31.2508   
     2        25.3240  29.3204  28.2928  30.0438  30.4477  30.9551  28.1232   
     3        32.2860  30.1546  28.6942  29.9960  29.6327  29.0865  30.5070   
2015 1        33.1515  29.9863  29.6798  28.3646  31.8518  31.5910  29.3723   
     2        27.9015  32.7687  30.7324  28.7988  29.4542  29.3931  30.4970   
     3        36.8335  29.0620  30.6849  29.8823  27.6963  30.9413  30.5530   

subject                         
type             Temp    Admin  
year session                    
2013 1        27.4274  30.3896  
     2        30.9259  30.1265  
     3        30.8675  30.7979  
2014 1        30.1317  29.0871  
     2        29.9151  29.2973  
     3        28.6844  29.6195  
2015 1        29.8012  30.3921  
     2        27.8455  30.5205  
     3        28.6712  28.8165

Remember that columns are primary in a ``DataFrame``, and the syntax used for multiply indexed ``Series`` applies to the columns.
For example, we can recover Guido's heart rate data with a simple operation:

In [79]:
health_data['Guido', 'HR']

year  session
2013  1          29.4441
      2          31.8484
      3          29.4538
2014  1          29.0034
      2          30.0438
      3          29.9960
2015  1          28.3646
      2          28.7988
      3          29.8823
Name: (Guido, HR), dtype: float64

Also, as with the single-index case, we can use the ``loc``, ``iloc``, and ``ix`` indexers introduced in [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb). For example:

In [90]:
health_data.iloc[0:9, 0:9]

subject           Bob                      Guido                        Sue  \
type               HR     Temp    Admin       HR     Temp    Admin       HR   
year session                                                                  
2013 1        36.8280  30.4323  30.2209  29.4441  29.7067  30.2776  30.2611   
     2        26.5965  29.9151  31.8368  31.8484  29.3796  29.0087  27.7863   
     3        39.4260  31.2263  29.1585  29.4538  28.6172  28.9508  29.8821   
2014 1        32.1890  31.4605  30.1290  29.0034  28.7336  29.2168  31.2508   
     2        25.3240  29.3204  28.2928  30.0438  30.4477  30.9551  28.1232   
     3        32.2860  30.1546  28.6942  29.9960  29.6327  29.0865  30.5070   
2015 1        33.1515  29.9863  29.6798  28.3646  31.8518  31.5910  29.3723   
     2        27.9015  32.7687  30.7324  28.7988  29.4542  29.3931  30.4970   
     3        36.8335  29.0620  30.6849  29.8823  27.6963  30.9413  30.5530   

subject                         
type             Temp    Admin  
year session                    
2013 1        27.4274  30.3896  
     2        30.9259  30.1265  
     3        30.8675  30.7979  
2014 1        30.1317  29.0871  
     2        29.9151  29.2973  
     3        28.6844  29.6195  
2015 1        29.8012  30.3921  
     2        27.8455  30.5205  
     3        28.6712  28.8165

These indexers provide an array-like view of the underlying two-dimensional data, but each individual index in ``loc`` or ``iloc`` can be passed a tuple of multiple indices. For example:

In [97]:
health_data.loc[:, ('Bob', 'HR')]

year  session
2013  1          36.8280
      2          26.5965
      3          39.4260
2014  1          32.1890
      2          25.3240
      3          32.2860
2015  1          33.1515
      2          27.9015
      3          36.8335
Name: (Bob, HR), dtype: float64

In [102]:
health_data.loc[(2015, 1),:]

subject  type 
Bob      HR       33.1515
         Temp     29.9863
         Admin    29.6798
Guido    HR       28.3646
         Temp     31.8518
         Admin    31.5910
Sue      HR       29.3723
         Temp     29.8012
         Admin    30.3921
Name: (2015, 1), dtype: float64

In [103]:
health_data.loc[(2015, 1),('Bob', 'HR')]

33.1515

Working with slices within these index tuples is not especially convenient; trying to create a slice within a tuple will lead to a syntax error:

In [104]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-104-fb34fa30ac09>, line 1)

You could get around this by building the desired slice explicitly using Python's built-in ``slice()`` function, but a better way in this context is to use an ``IndexSlice`` object, which Pandas provides for precisely this situation.
For example:

In [111]:
idx = pd.IndexSlice
health_data.loc[idx[:, 3], idx[:, 'Admin']]

,subject,Bob,Guido,Sue
,type,Admin,Admin,Admin
year,session,,,
2013,3,29.1585,28.9508,30.7979
2014,3,28.6942,29.0865,29.6195
2015,3,30.6849,30.9413,28.8165


In [110]:
health_data.loc[idx[:, 3], idx[:, :]]

subject           Bob                      Guido                        Sue  \
type               HR     Temp    Admin       HR     Temp    Admin       HR   
year session                                                                  
2013 3        39.4260  31.2263  29.1585  29.4538  28.6172  28.9508  29.8821   
2014 3        32.2860  30.1546  28.6942  29.9960  29.6327  29.0865  30.5070   
2015 3        36.8335  29.0620  30.6849  29.8823  27.6963  30.9413  30.5530   

subject                         
type             Temp    Admin  
year session                    
2013 3        30.8675  30.7979  
2014 3        28.6844  29.6195  
2015 3        28.6712  28.8165

There are so many ways to interact with data in multiply indexed ``Series`` and ``DataFrame``s, and as with many tools in this book the best way to become familiar with them is to try them out!

## Rearranging Multi-Indices

One of the keys to working with multiply indexed data is knowing how to effectively transform the data.
There are a number of operations that will preserve all the information in the dataset, but rearrange it for the purposes of various computations.
We saw a brief example of this in the ``stack()`` and ``unstack()`` methods, but there are many more ways to finely control the rearrangement of data between hierarchical indices and columns, and we'll explore them here.

### Sorted and unsorted indices

Earlier, we briefly mentioned a caveat, but we should emphasize it more here.
*Many of the ``MultiIndex`` slicing operations will fail if the index is not sorted.*
Let's take a look at this here.

We'll start by creating some simple multiply indexed data where the indices are *not lexographically sorted*:

In [112]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.282262
      2      0.951352
c     1      0.161174
      2      0.736342
b     1      0.886701
      2      0.592306
dtype: float64

If we try to take a partial slice of this index, it will result in an error:

In [113]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


Although it is not entirely clear from the error message, this is the result of the MultiIndex not being sorted.
For various reasons, partial slices and other similar operations require the levels in the ``MultiIndex`` to be in sorted (i.e., lexographical) order.
Pandas provides a number of convenience routines to perform this type of sorting; examples are the ``sort_index()`` and ``sortlevel()`` methods of the ``DataFrame``.
We'll use the simplest, ``sort_index()``, here:

In [114]:
data = data.sort_index()
data

char  int
a     1      0.282262
      2      0.951352
b     1      0.886701
      2      0.592306
c     1      0.161174
      2      0.736342
dtype: float64

With the index sorted in this way, partial slicing will work as expected:

In [115]:
data['a':'b']

char  int
a     1      0.282262
      2      0.951352
b     1      0.886701
      2      0.592306
dtype: float64

### Stacking and unstacking indices

As we saw briefly before, it is possible to convert a dataset from a stacked multi-index to a simple two-dimensional representation, optionally specifying the level to use:

In [119]:
pop.unstack(level=0)
health_data.unstack(level=0)

subject      Bob                                                       \
type          HR                      Temp                      Admin   
year        2013    2014     2015     2013     2014     2015     2013   
session                                                                 
1        36.8280  32.189  33.1515  30.4323  31.4605  29.9863  30.2209   
2        26.5965  25.324  27.9015  29.9151  29.3204  32.7687  31.8368   
3        39.4260  32.286  36.8335  31.2263  30.1546  29.0620  29.1585   

subject                      Guido  ...               Sue                    \
type                            HR  ...    Admin       HR                     
year        2014     2015     2013  ...     2015     2013     2014     2015   
session                             ...                                       
1        30.1290  29.6798  29.4441  ...  31.5910  30.2611  31.2508  29.3723   
2        28.2928  30.7324  31.8484  ...  29.3931  27.7863  28.1232  30.4970   
3        28.6942  30.6849  29.4538  ...  30.9413  29.8821  30.5070  30.5530   

subject                                                        
type        Temp                      Admin                    
year        2013     2014     2015     2013     2014     2015  
session                                                        
1        27.4274  30.1317  29.8012  30.3896  29.0871  30.3921  
2        30.9259  29.9151  27.8455  30.1265  29.2973  30.5205  
3        30.8675  28.6844  28.6712  30.7979  29.6195  28.8165  

[3 rows x 27 columns]

In [123]:
pop.unstack(level=1)
health_data.unstack(level=1)

subject      Bob                                                        \
type          HR                       Temp                      Admin   
session        1        2        3        1        2        3        1   
year                                                                     
2013     36.8280  26.5965  39.4260  30.4323  29.9151  31.2263  30.2209   
2014     32.1890  25.3240  32.2860  31.4605  29.3204  30.1546  30.1290   
2015     33.1515  27.9015  36.8335  29.9863  32.7687  29.0620  29.6798   

subject                      Guido  ...               Sue                    \
type                            HR  ...    Admin       HR                     
session        2        3        1  ...        3        1        2        3   
year                                ...                                       
2013     31.8368  29.1585  29.4441  ...  28.9508  30.2611  27.7863  29.8821   
2014     28.2928  28.6942  29.0034  ...  29.0865  31.2508  28.1232  30.5070   
2015     30.7324  30.6849  28.3646  ...  30.9413  29.3723  30.4970  30.5530   

subject                                                        
type        Temp                      Admin                    
session        1        2        3        1        2        3  
year                                                           
2013     27.4274  30.9259  30.8675  30.3896  30.1265  30.7979  
2014     30.1317  29.9151  28.6844  29.0871  29.2973  29.6195  
2015     29.8012  27.8455  28.6712  30.3921  30.5205  28.8165  

[3 rows x 27 columns]

The opposite of ``unstack()`` is ``stack()``, which here can be used to recover the original series:

In [121]:
pop.unstack().stack()
health_data.unstack().stack()

subject           Bob                      Guido                        Sue  \
type            Admin       HR     Temp    Admin       HR     Temp    Admin   
year session                                                                  
2013 1        30.2209  36.8280  30.4323  30.2776  29.4441  29.7067  30.3896   
     2        31.8368  26.5965  29.9151  29.0087  31.8484  29.3796  30.1265   
     3        29.1585  39.4260  31.2263  28.9508  29.4538  28.6172  30.7979   
2014 1        30.1290  32.1890  31.4605  29.2168  29.0034  28.7336  29.0871   
     2        28.2928  25.3240  29.3204  30.9551  30.0438  30.4477  29.2973   
     3        28.6942  32.2860  30.1546  29.0865  29.9960  29.6327  29.6195   
2015 1        29.6798  33.1515  29.9863  31.5910  28.3646  31.8518  30.3921   
     2        30.7324  27.9015  32.7687  29.3931  28.7988  29.4542  30.5205   
     3        30.6849  36.8335  29.0620  30.9413  29.8823  27.6963  28.8165   

subject                         
type               HR     Temp  
year session                    
2013 1        30.2611  27.4274  
     2        27.7863  30.9259  
     3        29.8821  30.8675  
2014 1        31.2508  30.1317  
     2        28.1232  29.9151  
     3        30.5070  28.6844  
2015 1        29.3723  29.8012  
     2        30.4970  27.8455  
     3        30.5530  28.6712

### Index setting and resetting

Another way to rearrange hierarchical data is to turn the index labels into columns; this can be accomplished with the ``reset_index`` method.
Calling this on the population dictionary will result in a ``DataFrame`` with a *state* and *year* column holding the information that was formerly in the index.
For clarity, we can optionally specify the name of the data for the column representation:

In [124]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


Often when working with data in the real world, the raw input data looks like this and it's useful to build a ``MultiIndex`` from the column values.
This can be done with the ``set_index`` method of the ``DataFrame``, which returns a multiply indexed ``DataFrame``:

In [125]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

In practice, I find this type of reindexing to be one of the more useful patterns when encountering real-world datasets.

## Data Aggregations on Multi-Indices

We've previously seen that Pandas has built-in data aggregation methods, such as ``mean()``, ``sum()``, and ``max()``.
For hierarchically indexed data, these can be passed a ``level`` parameter that controls which subset of the data the aggregate is computed on.

For example, let's return to our health data:

In [126]:
health_data

subject           Bob                      Guido                        Sue  \
type               HR     Temp    Admin       HR     Temp    Admin       HR   
year session                                                                  
2013 1        36.8280  30.4323  30.2209  29.4441  29.7067  30.2776  30.2611   
     2        26.5965  29.9151  31.8368  31.8484  29.3796  29.0087  27.7863   
     3        39.4260  31.2263  29.1585  29.4538  28.6172  28.9508  29.8821   
2014 1        32.1890  31.4605  30.1290  29.0034  28.7336  29.2168  31.2508   
     2        25.3240  29.3204  28.2928  30.0438  30.4477  30.9551  28.1232   
     3        32.2860  30.1546  28.6942  29.9960  29.6327  29.0865  30.5070   
2015 1        33.1515  29.9863  29.6798  28.3646  31.8518  31.5910  29.3723   
     2        27.9015  32.7687  30.7324  28.7988  29.4542  29.3931  30.4970   
     3        36.8335  29.0620  30.6849  29.8823  27.6963  30.9413  30.5530   

subject                         
type             Temp    Admin  
year session                    
2013 1        27.4274  30.3896  
     2        30.9259  30.1265  
     3        30.8675  30.7979  
2014 1        30.1317  29.0871  
     2        29.9151  29.2973  
     3        28.6844  29.6195  
2015 1        29.8012  30.3921  
     2        27.8455  30.5205  
     3        28.6712  28.8165

Perhaps we'd like to average-out the measurements in the two visits each year. We can do this by naming the index level we'd like to explore, in this case the year:

In [130]:
data_mean = health_data.mean(level='year')
data_mean

subject        Bob                            Guido                        \
type            HR       Temp      Admin         HR       Temp      Admin   
year                                                                        
2013     34.283500  30.524567  30.405400  30.248767  29.234500  29.412367   
2014     29.933000  30.311833  29.038667  29.681067  29.604667  29.752800   
2015     32.628833  30.605667  30.365700  29.015233  29.667433  30.641800   

subject        Sue                        
type            HR       Temp      Admin  
year                                      
2013     29.309833  29.740267  30.438000  
2014     29.960333  29.577067  29.334633  
2015     30.140767  28.772633  29.909700

By further making use of the ``axis`` keyword, we can take the mean among levels on the columns as well:

In [131]:
data_mean.mean(axis=1, level='type')

type,HR,Temp,Admin
year,,,
2013,31.280700,29.833111,30.085256
2014,29.858133,29.831189,29.375367
2015,30.594944,29.681911,30.305733


Thus in two lines, we've been able to find the average heart rate and temperature measured among all subjects in all visits each year.
This syntax is actually a short cut to the ``GroupBy`` functionality, which we will discuss in [Aggregation and Grouping](03.08-Aggregation-and-Grouping.ipynb).
While this is a toy example, many real-world datasets have similar hierarchical structure.

## Aside: Panel Data

Pandas has a few other fundamental data structures that we have not yet discussed, namely the ``pd.Panel`` and ``pd.Panel4D`` objects.
These can be thought of, respectively, as three-dimensional and four-dimensional generalizations of the (one-dimensional) ``Series`` and (two-dimensional) ``DataFrame`` structures.
Once you are familiar with indexing and manipulation of data in a ``Series`` and ``DataFrame``, ``Panel`` and ``Panel4D`` are relatively straightforward to use.
In particular, the ``ix``, ``loc``, and ``iloc`` indexers discussed in [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb) extend readily to these higher-dimensional structures.

We won't cover these panel structures further in this text, as I've found in the majority of cases that multi-indexing is a more useful and conceptually simpler representation for higher-dimensional data.
Additionally, panel data is fundamentally a dense data representation, while multi-indexing is fundamentally a sparse data representation.
As the number of dimensions increases, the dense representation can become very inefficient for the majority of real-world datasets.
For the occasional specialized application, however, these structures can be useful.
If you'd like to read more about the ``Panel`` and ``Panel4D`` structures, see the references listed in [Further Resources](03.13-Further-Resources.ipynb).

<!--NAVIGATION-->
< [Handling Missing Data](03.04-Missing-Values.ipynb) | [Contents](Index.ipynb) | [Combining Datasets: Concat and Append](03.06-Concat-And-Append.ipynb) >